# Finding clusters in data generated by the HIV branching process simulator

This notebook shows how to compute clusters using the code at [https://github.com/ghart-IDM/HIVclusteringExample/blob/clustering/fall2022Review/getClusters.py](https://github.com/ghart-IDM/HIVclusteringExample/blob/clustering/fall2022Review/getClusters.py).

Let's start by loading the requierd packages, including IDM's phylomodels as well as the branching process simulator that is coded in R:

In [1]:
# Standard packages
import os
import numpy as np
import pandas as pd

%matplotlib inline

# IDM's phylomodels
from phylomodels.trees import generate_treeFromFile
from phylomodels.trees.transform_transToPhyloTree import transform_transToPhyloTree
from phylomodels.trees.transform_joinTrees import transform_joinTrees

# R-related packages
import rpy2
import rpy2.robjects as robjects
r = robjects.r

from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# We may need to install some packages
try:
    from rpy2.robjects.packages import importr
    dplyr = importr('dplyr')
except RRuntimeError:
    from rpy2.robjects.packages import importr, data
    utils = importr('utils')
    base = importr('base')
    utils.chooseCRANmirror()
    utils.install_packages('dplyr')

# Set up working directory
cwd = os.getcwd()
os.chdir('../')

Now let's run a simulation and clean up its output:

In [2]:
r.source('hiv_branching_process.R')

value,[RTYPES.NILSXP]
visible,[RTYPES.LGLSXP]


In [3]:
out = r.simulate_transmission( sim_time=365*10 )
population_summary_r  = out.rx2('population_summary' )
transmission_record_r = out.rx2('transmission_record')

with localconverter( robjects.default_converter + pandas2ri.converter ):
    population_summary  = robjects.conversion.rpy2py( population_summary_r  )
    transmission_record = robjects.conversion.rpy2py( transmission_record_r )

  |==================================================| 100%


In [4]:
population_summary['recipient'] = population_summary['recipient'].astype(int).astype(str)
population_summary['source'] = population_summary['source'].astype(int).astype(str)

# Let's get rid of seed infections that didn't produce other infections
all_seeds = population_summary[ population_summary['source']=='0' ].index
all_infected_by_seed = population_summary[ population_summary['source'].isin( all_seeds ) ]
all_successful_seeds = all_infected_by_seed['source'].unique()
all_unsuccessful_seeds = list( set(all_seeds) - set(all_successful_seeds) )
population_summary_lean = population_summary.drop( index=all_unsuccessful_seeds )

We can build the transmission tree:

In [5]:
trees = generate_treeFromFile.read_treeFromLineList( population_summary,
                                                     ID = 'recipient',
                                                     infectorID = 'source',
                                                     infectTime = 'infectionTime',
                                                     sampleTime = 'sampleTime',
                                                     features = ['partners', 'acts_per_day', 'transmission_risk_per_act', 'removal_rate']
                                                   )
raw_tree = trees[0]

seeds = raw_tree.get_children()
seeds_phylo = []
for this_seed in seeds:
    seed_tree = transform_transToPhyloTree( this_seed )
    seeds_phylo.append( seed_tree )

tree = transform_joinTrees( seeds_phylo )
tree.describe()

Number of leaf nodes:	4260
Total number of nodes:	8519
Rooted:	Yes
Most distant node:	4012
Max. distance:	4351.307646


And finally, let's compute clustering metrics.

First, let's code some utility functions (copied from [https://github.com/ghart-IDM/HIVclusteringExample/blob/clustering/fall2022Review/getClusters.py](https://github.com/ghart-IDM/HIVclusteringExample/blob/clustering/fall2022Review/getClusters.py):

In [6]:
def walk_up (nodes, curnode, pathlen, topology_only=False):
    """
    Recursive function for traversing up a tree.
    """
    if topology_only:
        pathlen += 1
        if curnode.is_leaf():
            nodes.append( (curnode.name, pathlen) )
        else:
            nodes.append( (curnode.name, pathlen) )
            for c in curnode.children:
                nodes = walk_up(nodes, c, pathlen, topology_only)
    else:
        pathlen += curnode.dist
        if curnode.is_leaf():
            nodes.append( (curnode.name, pathlen) )
        else:
            nodes.append( (curnode.name, pathlen) )
            for c in curnode.children:
                nodes = walk_up(nodes, c, pathlen, topology_only)

    return nodes


def find_short_edges(tree, topology_only=False):
    """
    Find the shortest edge from the earliest sequence of a patient to a
    any sequence from any other patient.
    minimize = keep only edge from earliest seq to the closest other seq
    keep_ties = [to be used in conjunction with minimize]
                report all edges with the same minimum distance
    """

    # generate dictionary of child->parent associations
    parents = {}
    names = []
    for clade in tree.traverse('levelorder'):
        names.append(clade.name)
        for child in clade.children:
            parents.update({child: clade})

    nodes = tree.get_descendants('levelorder')
    num_nodes = len(nodes)
    distance = np.zeros((num_nodes+1, num_nodes+1), dtype=np.float32)
    index = {tree.name: 0}
    count = 1
    for node in nodes:
        index.update({node.name: count})
        count += 1
        
    # Get distances from root
    node2 = walk_up([], tree, {True: -1, False: -tree.dist}[topology_only], topology_only)
    
    i = index[tree.name]
    for n2, dist in node2:
        j = index[n2]
        distance[i, j] = dist
    distance[i, i] = 0
    
    for node in nodes:
        i = index[node.name]
        j = index[parents[node].name]
        dist = {True: 1, False: node.dist}[topology_only]
        distance[i, :] = distance[j, :] + dist
        node2 = walk_up([], node, {True: -1, False: -node.dist}[topology_only], topology_only)
        
        
        for n2, dist in node2:
            j = index[n2]
            distance[i, j] = dist
        distance[i, i] = 0
            
    return distance, names
        

The clustering metrics:

In [7]:
from scipy.sparse.csgraph import connected_components

# Get nearest neighbor list
distance, names = find_short_edges(tree)
leaves = tree.get_leaf_names()
names = np.array(names)
print(len(names), 'number of nodes')
print(len(leaves), 'number of leaves')
idx = np.in1d(names, leaves)
distance = distance[idx,:][:,idx]
np.fill_diagonal(distance, float('inf'))
names = names[idx]
    
# Calculate clusters
cutoffs = np.multiply( 100, [0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0] )
for i, cutoff in enumerate(cutoffs):
    num_clusters, cluster_labels = connected_components(distance < cutoff)
    cluster_labels = cluster_labels.tolist()

    cluster_sizes = { str(i): cluster_labels.count(i) for i in cluster_labels }
    cluster_size_distribution = {}
    for cluster_id, size in cluster_sizes.items():
        if size != 1:  # we are interested in clusters, not singletons
            cluster_size_distribution[str(size)] = 1 + cluster_size_distribution.get( str(size), 0 )
    
    n_clusters = 0
    if len(cluster_size_distribution) != 0:
        cluster_size_sum = 0
        for key, value in cluster_size_distribution.items():
            n_clusters += value
            cluster_size_sum += int(key)*value
        avg_cluster_size = ( cluster_size_sum/n_clusters )
    else:
        avg_cluster_size = 0

    print( '{:2} clusters of average size {:.2f} for cutoff {}'.format(n_clusters, avg_cluster_size, cutoff) )

8519 number of nodes
4260 number of leaves
16 clusters of average size 2.31 for cutoff 50.0
21 clusters of average size 2.62 for cutoff 100.0
18 clusters of average size 3.56 for cutoff 200.0
19 clusters of average size 3.58 for cutoff 300.0
18 clusters of average size 3.94 for cutoff 400.0
17 clusters of average size 4.53 for cutoff 500.0
15 clusters of average size 6.00 for cutoff 600.0
13 clusters of average size 9.69 for cutoff 700.0
780 clusters of average size 4.87 for cutoff 800.0
547 clusters of average size 7.38 for cutoff 900.0
450 clusters of average size 9.19 for cutoff 1000.0
